### Importing needed libraries

In [1]:
import pandas as pd
import numpy as np
import random
random.seed(1)

### Creating a list with spending categories

In [3]:
descriptions_list = {'Netflix':1,'Zabka':8,'Rossman':2,'Lidl':3,'H&M':0.5,'Zara':0.3,'Amazon':3,'PysznePL':7,
              'Kino':1,'Ubezpieczenie':0.09,'Orlen':3,'PKP':1,'Rent':1,'Doctor':0.2,'Gym':1, 'Phone':1,
             'Bills':1,'Steam':0.5,'Origin':0.2, 'Gessler':0.05,'Dentist':0.08,'Spotify':1}
#key = company name; value = frequency(how many times a month is a person likely to make transaction at given shop/site)
population_d = list(descriptions_list.keys())
weights_d = list(descriptions_list.values())
descriptions = random.choices(population_d , weights_d, k=1500)

### Creating dates for transactions

In [4]:
dates = pd.Series(pd.date_range("2018", freq="D", periods=1500))

### Creating expenses

In [5]:
tiers = {'Netflix':1,'Zabka':2,'Rossman':2,'Lidl':2,'H&M':2,'Zara':2,'Amazon':3,'PysznePL':1,'Kino':1,'Ubezpieczenie':3,
         'Orlen':2,'PKP':1,'Rent':3,'Doctor':2,'Gym':1, 'Phone':1, 'Bills':3,'Steam':2,'Origin':2, 
         'Gessler':2,'Dentist':2,'Spotify':1}

In [6]:
def kwota(tier):
    if tier == 1:
        kwt = random.randrange(50)
    elif tier == 2:
        kwt = random.randrange(50,200)
    elif tier == 3:
        kwt = random.randrange(200,500)
    return kwt

In [7]:
x = [kwota(tiers[d]) for d in descriptions]

In [8]:
sample_dataset = pd.DataFrame({'Opis':descriptions,'Kwota':x}, index=dates)
sample_dataset.index.name ='Data'
sample_dataset['Kwota']=sample_dataset['Kwota']*-1
sample_dataset

,Opis,Kwota
Data,,
2018-01-01,Zabka,-102
2018-01-02,Rent,-478
2018-01-03,Orlen,-69
2018-01-04,Rossman,-190
2018-01-05,Amazon,-361
...,...,...
2022-02-04,Zabka,-65
2022-02-05,Rossman,-167
2022-02-06,Amazon,-256


### Creating incomes

In [9]:
base_income = sample_dataset.groupby([(sample_dataset.index.year), (sample_dataset.index.month)]).sum().min() * -1 + 300
#I want the income to always exceed monthly spendings.

In [10]:
base_salary = float(base_income)
salaries = [base_salary * (1+i*0.01) for i in range(50)]
dates_s = pd.Series(pd.date_range("2018", freq="M", periods=1500//30))
bonus = [random.randrange(1000,2000) for i in range(0,1499,180)]
dates_b = []
for i in range(len(bonus)):
    index =  i * 180
    dates_b.append(dates[index])
salaries_df = pd.DataFrame({'Opis':"Przelew Przychodzący",'Kwota':salaries}, dates_s)
bonus_df = pd.DataFrame({'Opis':'Przelew Przychodzący','Kwota':bonus}, dates_b)
salaries_df.index.name = "Data"
bonus_df.index.name = "Data"

### Combining Data and Exporting

In [11]:
bank_Statement  = pd.concat([sample_dataset,salaries_df,bonus_df]).reset_index().sort_values(['Data','Kwota']).set_index('Data')
#bank_Statement.sort_index()
bank_Statement

,Opis,Kwota
Data,,
2018-01-01,Zabka,-102.00
2018-01-01,Przelew Przychodzący,1359.00
2018-01-02,Rent,-478.00
2018-01-03,Orlen,-69.00
2018-01-04,Rossman,-190.00
...,...,...
2022-02-05,Rossman,-167.00
2022-02-06,Amazon,-256.00
2022-02-07,Orlen,-109.00


In [12]:
bank_Statement.to_csv('Bank_Statement.csv')